In [4]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language
from git import Repo
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from os import getenv

api_key = 'key'

In [5]:
# Clone
repo_path = "./repo"
repo = Repo.clone_from("https://github.com/staszczm/AIChallange-A.I.diots", to_path=repo_path)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/staszczm/AIChallange-A.I.diots ./repo
  stderr: 'fatal: destination path './repo' already exists and is not an empty directory.
'

In [7]:
# Load
loader = GenericLoader.from_filesystem(
    path=repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()
print(len(documents))

67


In [8]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
print(len(texts))

86


In [9]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=(), openai_api_key=api_key))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [10]:
# Initialize LangChain with the OpenAI model
lc = ChatOpenAI(
    model_name='gpt-4-1106-preview',
    openai_api_key = api_key,
)   
memory = ConversationSummaryMemory(
    llm=lc, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(lc, retriever=retriever, memory=memory)

In [11]:
question = "How can I initialize a ReAct agent?"
result = qa(question)
result["answer"]

"I'm sorry, but the provided context does not contain any information about initializing a React agent. It seems to be related to a Django application, which is a Python web framework, and does not pertain to React, which is a JavaScript library for building user interfaces. If you need assistance with initializing a React component or setting up a React application, please provide the relevant context or ask a question regarding React specifically."

In [12]:
question = "What is this project about?"
result = qa(question)
result["answer"]

'The context provided does not mention anything about a React agent or a React project. The provided code snippets and context are related to a Django web application. Django is a Python web framework used for building web applications. The code snippets indicate that the application likely pertains to a school management system, with models and views for handling academic sessions, terms, and site configurations.\n\nIf you are looking for information on how a React agent would be initialized within the context of a Django project, it would typically involve setting up a React frontend application that communicates with the Django backend through API endpoints. However, there is no mention of React or such an integration within the provided context.'

In [13]:
question = "Describe in details just the pieces of code that cover Django"
result = qa(question)
result["answer"]

'Certainly! The provided code snippets come from a Django project, which appears to be for a school management system. Django is a high-level Python web framework that encourages rapid development and clean, pragmatic design. Here\'s a detailed breakdown of the code and its purpose within the Django framework:\n\n1. Django Test:\n   ```python\n   from django.test import TestCase\n   ```\n   This import statement brings in Django\'s TestCase class, which is a subclass of `unittest.TestCase` and is used for writing tests for Django applications.\n\n2. Django Forms:\n   ```python\n   from django.forms import inlineformset_factory, modelformset_factory\n   ```\n   These are functions that help create formsets associated with model instances. `inlineformset_factory` creates a formset that is connected to a parent model, while `modelformset_factory` creates a formset that is connected to a model.\n\n3. Model Formsets:\n   ```python\n   InvoiceItemFormset = inlineformset_factory(\n       Invo

In [14]:
question = ("Do you have any idea on how to improve this project?")
result = qa(question)
print(f"""{result["answer"]}""")

The provided code snippets and context show a basic Django web application for a school management system with several models like `Student`, `AcademicSession`, `AcademicTerm`, `Subject`, `StudentClass`, and `SiteConfig`. The web application also includes views for listing and detailing students, creating students, and managing site configurations and academic sessions/terms.

To suggest improvements, we'll consider different aspects of the application:

1. **User Experience (UX) and Design:**
   - Improve the UI/UX design for better navigation and user engagement.
   - Implement responsive design for mobile and tablet users.
   - Provide a dashboard for a quick overview of key metrics (e.g., student count, outstanding fees).

2. **Features and Functionality:**
   - Add a module for attendance tracking.
   - Integrate a gradebook for teachers to record and calculate students' grades.
   - Implement a scheduling system for classes and exams.
   - Add a communication platform for announc